# 15行代码搞定Kaggle电影评论情感分析

>同济大学 张子豪 2019-6-28

>Bilibili视频：同济子豪兄

接下来的20分钟，我将手把手带你参加Kaggle数据科学竞赛：电影评论情感分析与文本数据挖掘，用不到20行代码超过百分之九十的参赛选手，进入前10%。
你将掌握文本数据预处理、去除停用词、词袋模型、TF-IDF模型等自然语言处理和文本数据挖掘的基础知识，并掌握数据集拆分、逻辑回归模型、超参数的网格搜索、交叉验证、模型效果评价等机器学习基础知识，了解参加Kaggle数据竞赛的流程。

通过本案例，我想告诉你三件事：

- Kaggle数据竞赛没什么可怕的，大部分的参赛选手都比较水。
- 黑猫白猫，抓到老鼠就是好猫。算法不是越复杂越好，有时简单的算法效果也很不错，甚至“一招逻辑回归包打天下”。
- 数据预处理和特征工程很重要，对于文本数据来说，有词袋模型、TF-IDF模型、word2vec模型等方法来抽取文本的特征。

如果你不太会使用现在看到的这个jupyter notebook工具，请看子豪兄的这个视频

[python数据分析神器Jupyter notebook快速入门](https://www.bilibili.com/video/av54100790)

# 数据集介绍

Kaggle竞赛网址：
https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews

![数据集介绍](https://upload-images.jianshu.io/upload_images/13714448-a8acedd302258ce7.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

![排行榜](https://upload-images.jianshu.io/upload_images/13714448-3955c5fc364d919d.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


![参赛队与一些公开的kernel](https://upload-images.jianshu.io/upload_images/13714448-21af581b730bf01c.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


# 下载所需工具包

在命令行界中运行下面这行命令，从清华大学开源软件镜像站下载pandas、sklearn这两个python第三方模块。

`pip install pandas sklearn -i https://pypi.tuna.tsinghua.edu.cn/simple`

如果命令行提示找不到pip命令，那说明你没有安装Python，请看子豪兄的视频
[一劳永逸安装和配置python3.7.2（新手必读）](https://www.bilibili.com/video/av45546100)

# 导入数据集

In [1]:
# pandas是用来导入、整理、清洗表格数据的专用工具，类似excel，但功能更加强大，导入的时候给pandas起个小名叫pd
import pandas as pd

In [2]:
# 用pandas的read_csv函数读取训练数据及测试数据，数据文件是.tsv格式的，也就是说数据用制表符\t分隔，类似于.csv文件的数据用逗号分隔
data_train = pd.read_csv('./train.tsv',sep='\t')
data_test = pd.read_csv('./test.tsv',sep='\t')

In [3]:
# 看训练集数据前5行，Phrase列为电影评论文本，Sentiment为情感标签
data_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


电影评论的情感标签

- 0 - negative
- 1 - somewhat negative
- 2 - neutral
- 3 - somewhat positive
- 4 - positive

In [4]:
# 共有156060行训练数据，每行数据都有短语ID、句子ID、文本内容、情感标签四列
data_train.shape

(156060, 4)

In [5]:
# 查看测试集数据前5行，Phrase列就是需要我们自己构建模型预测情感标签的文本
data_test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [6]:
# 共有66292行测试集数据，每个数据都有短语ID、句子ID、文本内容三列
data_test.shape

(66292, 3)

# 构建语料库

- 我们需要对文本进行一些处理，将原始文本中的每一个词变成计算机看得懂的向量，这一过程叫做文本的特征工程，非常重要。
- 有很多将词变成向量的方法，比如下面将要介绍的词袋模型、TF-IDF模型，以及视频中介绍的word2vec模型。
- 不管采用什么模型，我们都需要先把训练集和测试集中所有文本内容组合在一起，构建一个语料库。

In [7]:
# 提取训练集中的文本内容 
train_sentences = data_train['Phrase']

# 提取测试集中的文本内容
test_sentences = data_test['Phrase']

# 通过pandas的concat函数将训练集和测试集的文本内容合并到一起
sentences = pd.concat([train_sentences,test_sentences])

In [8]:
# 合并到一起的语料库共有222352行数据
sentences.shape

(222352,)

In [9]:
# 提取训练集中的情感标签，一共是156060个标签
label = data_train['Sentiment']

In [10]:
label.shape

(156060,)

In [11]:
#导入停词库，停词库中的词是一些废话单词和语气词，对情感分析没什么帮助
stop_words = open('./stop_words.txt',encoding='utf-8').read().splitlines()

In [12]:
# stop_words是一个列表，列表中每一个元素都是一个停用词
stop_words

["\ufeffain'",
 'happy',
 'isn',
 'ain',
 'al',
 'couldn',
 'didn',
 'doesn',
 'hadn',
 'hasn',
 'haven',
 'sn',
 'll',
 'mon',
 'shouldn',
 've',
 'wasn',
 'weren',
 'won',
 'wouldn',
 "'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'t",
 "'ve",
 'ZT',
 'ZZ',
 'a',
 "a's",
 'able',
 'about',
 'above',
 'abst',
 'accordance',
 'according',
 'accordingly',
 'across',
 'act',
 'actually',
 'added',
 'adj',
 'adopted',
 'affected',
 'affecting',
 'affects',
 'after',
 'afterwards',
 'again',
 'against',
 'ah',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'announce',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anymore',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'apparently',
 'appear',
 'appreciate',
 'appropriate',
 'approximately',
 'are',
 'area',
 'areas',
 'aren',
 "aren't",
 'arent',
 'arise',
 'around',
 'as',
 'aside',
 'ask',
 'asked',
 'asking',
 

# 使用词袋模型进行文本特征工程

![词袋模型](https://upload-images.jianshu.io/upload_images/13714448-63922809a80864ff.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


In [13]:
# 用sklearn库中的CountVectorizer构建词袋模型
# 词袋模型的详细介绍请看子豪兄的视频
# analyzer='word'指的是以词为单位进行分析，对于拉丁语系语言，有时需要以字母'character'为单位进行分析
# ngram指分析相邻的几个词，避免原始的词袋模型中词序丢失的问题
# max_features指最终的词袋矩阵里面包含语料库中出现次数最多的多少个词

from sklearn.feature_extraction.text import CountVectorizer
co = CountVectorizer(
    analyzer='word',
    ngram_range=(1,4),
    stop_words=stop_words,
    max_features=150000
)

In [14]:
# 使用语料库，构建词袋模型

co.fit(sentences)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=150000, min_df=1,
        ngram_range=(1, 4), preprocessor=None,
        stop_words=["\ufeffain'", 'happy', 'isn', 'ain', 'al', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'sn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', "'d", "'ll", "'m", "'re", "'s", "'t", "'ve", 'ZT', 'ZZ', 'a', "a's", 'able', 'about', 'above', 'abst', 'accordance', 'accor...', ',', '·', '￥', '……', '（', '）', '——', '、', '：', '；', '“', '’', '《', '》', '，', '。', '、', '？', '★ '],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [15]:
# 将训练集随机拆分为新的训练集和验证集，默认3:1,然后进行词频统计
# 在机器学习中，训练集相当于课后习题，用于平时学习知识。验证集相当于模拟考试，用于检验学习成果。测试集相当于高考，用于最终Kaggle竞赛打分。
# 新的训练集和验证集都来自于最初的训练集，都是有标签的。

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_sentences,label,random_state=1234)

- x_train 训练集数据 （相当于课后习题）
- x_test 验证集数据 （相当于模拟考试题）
- y_train 训练集标签 （相当于课后习题答案）
- y_test 验证集标签（相当于模拟考试题答案）

In [16]:
# 随便看训练集中的一个数据
x_train[1]

'A series of escapades demonstrating the adage that what is good for the goose'

In [17]:
# 用上面构建的词袋模型，把训练集和验证集中的每一个词都进行特征工程，变成向量

x_train = co.transform(x_train)
x_test = co.transform(x_test)

In [18]:
# 随便看训练集中的一个数据，它是150000列的稀疏矩阵

x_train[1]

<1x150000 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

# 构建分类器算法，对词袋模型处理后的文本进行机器学习和数据挖掘

## 逻辑回归分类器

![逻辑回归](https://upload-images.jianshu.io/upload_images/13714448-8a388dfa095ffe4c.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)



In [19]:
# 直接运行本单元格即可，本单元格代码的作用是：忽略下面代码执行过程中的版本警告等无用提示

import warnings 
warnings.filterwarnings('ignore')

In [20]:
from sklearn.linear_model import LogisticRegression
lg1 = LogisticRegression()
lg1.fit(x_train,y_train)
print('词袋方法进行文本特征工程，使用sklearn默认的逻辑回归分类器，验证集上的预测准确率:',lg1.score(x_test,y_test))

词袋方法进行文本特征工程，使用sklearn默认的逻辑回归分类器，验证集上的预测准确率: 0.6430603613994618


## 多项式朴素贝叶斯分类器

In [21]:
#引用朴素贝叶斯进行分类训练和预测
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(x_train,y_train)
print('词袋方法进行文本特征工程，使用sklearn默认的多项式朴素贝叶斯分类器，验证集上的预测准确率:',classifier.score(x_test,y_test))

词袋方法进行文本特征工程，使用sklearn默认的多项式朴素贝叶斯分类器，验证集上的预测准确率: 0.6084070229398949


多项式朴素贝叶斯分类器，训练速度很快，但准确率较低。

# 使用TF-IDF模型进行文本特征工程

TF值衡量了一个词出现的次数。<br>
IDF值衡量了这个词是不是烂大街。如果是the、an、a等烂大街的词，IDF值就会很低。<br>
两个值的乘积TF_IDF反映了一个词的出现带来的特异性信息。<br>
例如，“中国”、“功夫”这两个词也许会同时出现，但“中国”这个词在各个文档中都有出现，IDF值很低，因此TF_IDF值也很低。
而“功夫”这个词只在特定文档中出现，这个词能带来的“特异性”信息就会大很多。

\begin{aligned}
TF(“功夫")&=\frac{"功夫"这个词在当前文章中出现的次数}{"功夫"这个词在整个语料库中出现的次数}\\
\\
\\
IDF(“功夫")&=ln \frac{语料库的总文档数}{语料库中"功夫"出现的文档数}\\
\\
\\
TF\_IDF(“功夫")&=IF(“功夫") \times IDF(“功夫")
\end{aligned}

In [22]:
# 用sklearn库中的TfidfVectorizer构建TF-IDF模型
# TF-IDF模型的详细介绍请看子豪兄的视频
# analyzer='word'指的是以词为单位进行分析，对于拉丁语系语言，有时需要以字母'character'为单位进行分析
# ngram指分析相邻的几个词，避免原始的词袋模型中词序丢失的问题
# max_features指最终的词袋矩阵里面包含语料库中出现次数最多的多少个词

# TF-IDF模型是专门用来过滤掉烂大街的词的，所以不需要引入停用词stop_words

from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1,4),
    # stop_words=stop_words,
    max_features=150000
)

In [23]:
tf.fit(sentences)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=150000, min_df=1,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

类似上面的操作，拆分原始训练集为训练集和验证集，用TF-IDF模型对每一个词都进行特征工程，变成向量

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_sentences,label,random_state=1234)

In [25]:
x_train = tf.transform(x_train)
x_test = tf.transform(x_test)

In [26]:
x_train[1]

<1x150000 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

# 构建分类器算法，对TF-IDF模型处理后的文本进行机器学习和数据挖掘

## 朴素贝叶斯分类器

In [27]:
#引用朴素贝叶斯进行分类训练和预测
classifier = MultinomialNB()
classifier.fit(x_train,y_train)
print('TF-IDF方法进行文本特征工程，使用sklearn默认的多项式朴素贝叶斯分类器，验证集上的预测准确率:',classifier.score(x_test,y_test))

TF-IDF方法进行文本特征工程，使用sklearn默认的多项式朴素贝叶斯分类器，验证集上的预测准确率: 0.6045367166474432


## 逻辑回归分类器

In [28]:
# sklearn默认的逻辑回归模型
lg1 = LogisticRegression()
lg1.fit(x_train,y_train)
print('TF-IDF方法进行文本特征工程，使用sklearn默认的逻辑回归模型，验证集上的预测准确率:',lg1.score(x_test,y_test))

TF-IDF方法进行文本特征工程，使用sklearn默认的逻辑回归模型，验证集上的预测准确率: 0.6326541073945918


In [29]:
# C：正则化系数，C越小，正则化效果越强
# dual：求解原问题的对偶问题
lg2 = LogisticRegression(C=3, dual=True)
lg2.fit(x_train,y_train)
print('TF-IDF方法进行文本特征工程，使用增加了两个参数的逻辑回归模型，验证集上的预测准确率:',lg2.score(x_test,y_test))

TF-IDF方法进行文本特征工程，使用增加了两个参数的逻辑回归模型，验证集上的预测准确率: 0.6533384595668332


对比两个预测准确率可以看出，在逻辑回归中增加C和dual这两个参数可以提高验证集上的预测准确率，但如果每次都手动修改就太麻烦了。我们可以用sklearn提供的强大的网格搜索功能进行超参数的批量试验。<br>
搜索空间：C从1到9。对每一个C，都分别尝试dual为True和False的两种参数。<br>最后从所有参数中挑出能够使模型在验证集上预测准确率最高的。

In [30]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C':range(1,10),
             'dual':[True,False]
              }
lgGS = LogisticRegression()
grid = GridSearchCV(lgGS, param_grid=param_grid,cv=3,n_jobs=-1)
grid.fit(x_train,y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': range(1, 10), 'dual': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [31]:
grid.best_params_

{'C': 5, 'dual': True}

最后超参数搜索的结果是，C为5，dual为True，能够使逻辑回归模型在验证集上预测准确率最高。我们便采用这个最优参数，构建lg_final分类器，最终在验证集上预测正确率为0.655464。

In [36]:
lg_final = grid.best_estimator_

In [38]:
print('经过网格搜索，找到最优超参数组合对应的逻辑回归模型，在验证集上的预测准确率:',lg_final.score(x_test,y_test))

经过网格搜索，找到最优超参数组合对应的逻辑回归模型，在验证集上的预测准确率: 0.6546456491093169


# 对测试集的数据进行预测，提交Kaggle竞赛最终结果

In [39]:
# 查看测试集数据前5行，Phrase列就是需要我们自己构建模型预测情感标签的文本
data_test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [51]:
# 使用TF-IDF对测试集中的文本进行特征工程
test_X = tf.transform(data_test['Phrase'])

In [43]:
# 对测试集中的文本，使用lg_final逻辑回归分类器进行预测
predictions = lg_final.predict(test_X)

In [46]:
predictions

array([2, 2, 3, ..., 1, 1, 2], dtype=int64)

In [47]:
predictions.shape

(66292,)

In [48]:
# 将预测结果加在测试集中

data_test.loc[:,'Sentiment'] = predictions

In [49]:
data_test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,2
1,156062,8545,An intermittently pleasing but mostly routine ...,2
2,156063,8545,An,3
3,156064,8545,intermittently pleasing but mostly routine effort,2
4,156065,8545,intermittently pleasing but mostly routine,2


In [52]:
# 按Kaggle比赛官网上的要求整理成这样的格式

final_data = data_test.loc[:,['PhraseId','Sentiment']]

In [53]:
final_data.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,3
3,156064,2
4,156065,2


In [54]:
# 保存为.csv文件，即为最终结果

final_data.to_csv('final_data.csv',index=None)

# 扩展阅读

除了词袋模型和TF-IDF模型之外，还有其它文本数据特征工程的方法，比如word2vec。<br>
你可以在这个网站http://projector.tensorflow.org/ 
看到word2vec的可视化效果，虽然这个效果是降到三维之后的，但仍然能看到关联词之间的关系。

# 微信扫描二维码给我打赏哦，支持我们做出更多的人工智能和机器学习视频教程。

>Bilibili视频专栏：同济子豪兄

![子豪兄赞赏码.png](https://upload-images.jianshu.io/upload_images/13714448-7be6682a94b8211b.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)